# Ability - Wealth Modelling

Ability like IQ can be modelled as a normal distribution, however, wealth in society follows a power law. Why is this the case? Suppose if outcomes are path dependent, with the state in previous steps influencing the later steps, will the final distribution follow a power law? This notebook seeks to model this.

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math as math
from pylab import rcParams
import random

%matplotlib inline

## Initial Parameters

We have an initial population size of 100. We know that ability follows a normal distribution so we can draw values from np.random.norm. Let's consider 

In [7]:
pop_size = 1000
ability_init = np.random.normal(loc=0.0, scale=1.0, size=(pop_size))


[-3.33633575e-01  1.20990786e+00 -8.30221104e-01  2.72154348e-01
 -1.15378812e+00 -1.46713032e+00 -9.36530006e-01  2.37288948e-01
  9.45239090e-01  5.40177605e-01  6.12731606e-01  1.73851902e+00
  1.88982243e-01 -1.93449632e+00 -5.88267473e-01 -3.69777534e-03
 -4.59981179e-01 -1.36862988e+00  2.49342804e+00 -9.64772821e-01
 -4.28333243e-01 -1.52054761e+00  4.00967501e-01 -3.86352573e-01
 -8.91441840e-01 -1.49048154e+00 -5.31485227e-02  1.74696497e+00
  1.66512720e-01  2.96869233e-01 -2.03614566e+00  1.18479300e+00
 -4.79860452e-01 -1.44881996e+00  1.55861757e+00 -8.88214654e-02
 -6.86237377e-01 -1.10102543e+00  1.07954897e-01  1.57069828e+00
 -3.95995960e-01  3.67209411e-01  6.90056459e-01  1.42944427e+00
  8.70784344e-01 -6.23780137e-01  1.65856420e-01 -1.46607293e+00
  1.02033283e+00 -1.05674702e+00 -8.42534328e-01  2.72924641e-01
  1.03055935e+00 -1.60370471e+00  3.22333064e-01  1.70963483e+00
  2.19942616e-02 -1.49189114e+00  3.52696824e-01  8.37166970e-01
  1.98793157e-01  5.46790